In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, tensorflow as tf, os
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers as tfl, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import cv2
import sys
from tensorflow.keras.utils import to_categorical
import os
from PIL import Image

In [23]:

# Ruta al directorio donde se guardó el modelo
PATH_TO_MODEL_DIR = '/content/drive/MyDrive/RedNeuronal_ResNet/modelsV4/saved_model/'

# Cargar el modelo
new_model = tf.keras.models.load_model(PATH_TO_MODEL_DIR)

# Definir las rutas de los datos y las imágenes
csv_path = '/content/drive/MyDrive/ModeloCompleto_Monografia/Coords_For_Classification/coords.csv'
img_dir = '/content/drive/MyDrive/ModeloCompleto_Monografia/Input_Classification/'
model_output_dir = '/content/drive/MyDrive/ModeloCompleto_Monografia/Eval_CompleteModel/output.csv'
# Obtener la lista de archivos de imágenes en el directorio
file_list = os.listdir(img_dir)

In [24]:
def load_image(image_path):
    image = Image.open(image_path)
    image = image.resize((64, 64))  # Asegúrate de que el tamaño de la imagen coincida con el modelo
    image_np = np.array(image)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    return input_tensor

def get_classification(output):
    max_index = np.argmax(output)
    labels = ["Star", "Galaxy"]

    # Obtener la clasificación asociada al índice
    classification = labels[max_index]

    return classification

In [25]:
# Cargar y procesar las imágenes
data = []
labels = []
for image_file in file_list:
    image_path = os.path.join(img_dir, image_file)
    input_tensor = load_image(image_path)
    data.append(input_tensor)
    labels.append(image_file)

In [26]:
# Realizar la clasificación para cada imagen
classifications = []
for input_tensor in data:
    result = new_model.predict(input_tensor)
    classifications.append(result)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 57ms/step


In [27]:
coords = pd.DataFrame(pd.read_csv(csv_path))
coords

,original_filename,object_name,xmin,ymin,xmax,ymax,detection_score
0,image_1001.jpg,objeto_1.jpg,294.217316,218.922058,304.049438,229.287277,0.687477
1,image_1001.jpg,objeto_2.jpg,19.905863,246.371033,29.520864,256.390656,0.579624
2,image_1001.jpg,objeto_3.jpg,0.746979,87.623604,11.053849,98.216927,0.551638
3,image_1001.jpg,objeto_4.jpg,196.626511,314.506042,206.510437,319.813354,0.526203
4,image_1001.jpg,objeto_5.jpg,114.885254,174.970428,124.434677,185.007660,0.434896
...,...,...,...,...,...,...,...
10587,image_1426.jpg,objeto_10588.jpg,120.901123,120.995499,130.238251,131.191528,0.326942
10588,image_1426.jpg,objeto_10589.jpg,101.266479,37.562813,111.227280,47.297356,0.325855
10589,image_1426.jpg,objeto_10590.jpg,101.036423,34.343536,110.978844,44.351341,0.318950
10590,image_1426.jpg,objeto_10591.jpg,222.547165,58.991150,232.363953,68.702934,0.315149


In [28]:
output_resnet = []
for image_index in range(len(data)):
  image_name = labels[image_index]
  classification = get_classification(classifications[image_index])
  output_resnet.append([image_name, classification])
output_resnet = pd.DataFrame(output_resnet,columns=['object_name','class'])
coords = coords.set_index('object_name').join(output_resnet.set_index('object_name'))
output_modelo = coords.copy()
output_modelo.to_csv(model_output_dir)

In [29]:
output_modelo

,original_filename,xmin,ymin,xmax,ymax,detection_score,class
object_name,,,,,,,
objeto_1.jpg,image_1001.jpg,294.217316,218.922058,304.049438,229.287277,0.687477,Galaxy
objeto_2.jpg,image_1001.jpg,19.905863,246.371033,29.520864,256.390656,0.579624,Galaxy
objeto_3.jpg,image_1001.jpg,0.746979,87.623604,11.053849,98.216927,0.551638,Star
objeto_4.jpg,image_1001.jpg,196.626511,314.506042,206.510437,319.813354,0.526203,Galaxy
objeto_5.jpg,image_1001.jpg,114.885254,174.970428,124.434677,185.007660,0.434896,Galaxy
...,...,...,...,...,...,...,...
objeto_10588.jpg,image_1426.jpg,120.901123,120.995499,130.238251,131.191528,0.326942,Galaxy
objeto_10589.jpg,image_1426.jpg,101.266479,37.562813,111.227280,47.297356,0.325855,Galaxy
objeto_10590.jpg,image_1426.jpg,101.036423,34.343536,110.978844,44.351341,0.318950,Galaxy
